In [ ]:
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cvxopt

import numpy as np

from cvxopt import matrix, spmatrix, normal, setseed, blas, lapack, solvers
import nucnrm
import scipy.stats 

In [ ]:
N = 2

M_1 = np.random.rand(4*N,2*N)
M_2 = np.random.rand(4*N,2*N)


In [ ]:
n_col = M_1.shape[1]
assert M_1.shape[1]==M_2.shape[1], "The two matrices have different number of columns"
n_row1 = M_1.shape[0]
n_row2 = M_2.shape[0]
n_row = n_row1+n_row2


"""
construct A: 
put together columnvectors 
each of them is a column extracted form M1 or M2


usually the vectors are column-major
if p <q we have to transpose the mathcal{A}
this means basically that the ordering becomes row major


"""
size_mat = n_row*n_col*2

A = np.zeros((size_mat,2*n_col))

if n_row>=n_col*2:#n>=q:
    for i in range(n_col):
        B = cvxopt.spmatrix([],[],[],size=(n_row,2*n_col))
        offset = n_row*i #elemets of the previous columns
        #set i-th collumn of M1. place it in i-th column of A
        A[offset:offset+n_row1,i]=M_1[:,i]
        offset += n_row*n_col  #elemets of the previous columns + offset for M2
        #set i-th collumn of M2. place it i-th column of A
        A[offset+n_row1:offset+n_row,n_col+i]=M_2[:,i]
else:
    for i in range(n_col):
        B = cvxopt.spmatrix([],[],[],size=(2*n_col,n_row))
        #set i-th collumn of M1. place it in i-th column of A
        A[i:n_col*2*n_row1:n_col*2,i]=M_1[:,i]

        #set i-th collumn of M2. place it i-th column of A
        A[i+n_col*2*n_row1+n_col::n_col*2,n_col+i]=M_2[:,i]    

In [ ]:
#short test
if n_row>=n_col*2:
    print("standard")
    mat = np.sum(A,axis = 1).reshape(n_col*2,n_row).T
    print(mat-block_diag(M_1,M_2))
else:
    print("transposed")
    mat = np.sum(A,axis = 1).reshape(n_row,n_col*2)
    print(mat-block_diag(M_1,M_2))
A.shape

In [ ]:
h = np.hstack([-np.ones(n_col),np.ones(n_col),np.zeros(2*n_col)])
h

In [ ]:
I = cvxopt.spmatrix(1.0, range(n_col), range(n_col))
G = cvxopt.sparse([cvxopt.sparse([[-I,I],[I,I]]),cvxopt.spdiag([I,I])])
print(G)

In [ ]:
sol = nucnrm.nrmapp(matrix(A), B,G = G, h = matrix(h))

In [ ]:
sol

## Second try:

I try to get rid of the additional x-dims.
This is done by setting

$$B = \begin{bmatrix} 0& \\ & M_2 \end{bmatrix}$$

And we only have $\text{dim}(x) = \text{# of columns}$. For each $x_i$ we have a 
$$A_i =
\begin{bmatrix}
0M_1[:,i] 0 & 0\\
0 & 0-M_2[:,i] 0
\end{bmatrix}
$$

we bound each $x$ to $0\leq x_i \leq1$.
If $x_i=1$ the vector in the second block is removed due to the negative sign.



In [ ]:
N = 3


M_1 = np.random.rand(N,2*N)
M_2 = np.random.rand(N,2*N)

M_1 = np.random.rand(2*N,N)
M_2 = np.random.rand(2*N,N)

#
k = np.ones(N*4,dtype=int)
k[:N]=0
k =np.random.permutation(k)

Q = scipy.stats.ortho_group.rvs(2*N)
A = Q[:,k]+1e-3*np.random.rand(2*N,4*N)

M_1 = A[:N,:]
M_2 = A[N:,:]

In [ ]:
n_col = M_1.shape[1]
assert M_1.shape[1]==M_2.shape[1], "The two matrices have different number of columns"
n_row1 = M_1.shape[0]
n_row2 = M_2.shape[0]
n_row = n_row1+n_row2


"""
construct A: 
put together columnvectors 
each of them is a column extracted form M1 or M2


usually the vectors are column-major
if p <q we have to transpose the mathcal{A}
this means basically that the ordering becomes row major


"""
size_mat = n_row*n_col*2

A = np.zeros((size_mat,n_col))

if n_row>=n_col*2:#n>=q:
    for i in range(n_col):
        B = cvxopt.matrix(block_diag(0*M_1,M_2))
        offset = n_row*i #elemets of the previous columns
        #set i-th collumn of M1. place it in i-th column of A
        A[offset:offset+n_row1,i]=M_1[:,i]
        offset += n_row*n_col  #elemets of the previous columns + offset for M2
        #set i-th collumn of M2. place it i-th column of A
        A[offset+n_row1:offset+n_row,i]=-M_2[:,i]
else:
    for i in range(n_col):
        B = cvxopt.matrix(block_diag(0*M_1.T,M_2.T))
        #set i-th collumn of M1. place it in i-th column of A
        A[i:n_col*2*n_row1:n_col*2,i]=M_1[:,i]

        #set i-th collumn of M2. place it i-th column of A
        A[i+n_col*2*n_row1+n_col::n_col*2,i]=-M_2[:,i]   

In [ ]:
#short test
if n_row>=n_col*2:
    print("standard")
    mat = np.sum(A,axis = 1).reshape(n_col*2,n_row).T
    #this is not actually trasposed in the solver, but from reshape magic...
    print(mat-block_diag(M_1,M_2))
    B_ = cvxopt.matrix(block_diag(-1*M_1,M_2))
else:
    print("transposed")
    mat = np.sum(A,axis = 1).reshape(n_row,n_col*2)
    print(mat-block_diag(M_1,M_2))
    B_ = cvxopt.matrix(block_diag(-1*M_1.T,M_2.T))
print(A.shape)

sol = nucnrm.nrmapp(matrix(A), B_)
print("x",sol['x'])

### Constrints on $x$

$$x_1 \geq 0$$
$$x_1 \leq 1$$

the first is equivalent to $-x_1 \leq 0$-

This gives the combined constrints

$$G x = \leq h  \qquad 
\begin{bmatrix}
-I\\
I
\end{bmatrix} x 
\leq
\begin{bmatrix}
0\\
1
\end{bmatrix}
$$

In [ ]:
h = np.hstack([np.zeros(n_col),np.ones(n_col)])
I = cvxopt.spmatrix(1.0, range(n_col), range(n_col))
G = cvxopt.sparse([-I,I])
#print(G)

In [ ]:
sol = nucnrm.nrmapp(matrix(A), B,G = G, h = matrix(h))
print("x",sol['x'])

In [ ]:
import mosek